In [ ]:
from tqdm import tqdm
import time
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 数据集预处理 ##

In [ ]:
# 加载Excel文件
base_info = pd.read_csv('EnergyContest/BSinfo.csv')
cell_data = pd.read_csv('EnergyContest/CLdata.csv', parse_dates=['Time'])
energy_data = pd.read_csv('EnergyContest/ECdata.csv', parse_dates=['Time'])
power_consumption_prediction = pd.read_csv('EnergyContest/power_consumption_prediction.csv', parse_dates=['Time'])

# 将 CLdata 与 BSinfo 合并
Total = pd.merge(base_info,cell_data, on=['BS', 'CellName'], how='inner')

#将'Antennas','Bandwidth','Frequency'转化为独热码
Total_dummies = pd.get_dummies(Total, columns=['Antennas','Bandwidth','Frequency'])
#提取数据透视表
Total_pivot = pd.pivot_table(Total_dummies, 
                             values=[col for col in Total_dummies.columns if (col != 'BS') and (col != 'Time') and (col != 'Mode') and (col != 'RUType')], 
                             index=['Time', 'BS', 'Mode', 'RUType'], 
                             columns='CellName', 
                             fill_value=0)
Total_pivot.reset_index(inplace=True)

# 将'Mode','RUType'转化为独热码
Total_pivot.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in Total_pivot.columns]
Total_pivot = pd.get_dummies(Total_pivot, columns=['Mode','RUType'])

#添加输入特征:负载平均数,负载总和,ESMODE平均数,ESMODE总和
load_columns = [col for col in Total_pivot.columns if col.startswith('load')]
Total_pivot['load_avg'] = Total_pivot[load_columns].mean(axis=1)
Total_pivot['load_sum'] = Total_pivot[load_columns].sum(axis=1)

#限定ESMode如何求值 可以自行赋予各节能模式对应权重
ESMode_columns = [col for col in Total_pivot.columns if col.startswith('ESMode')]
Total_pivot['ESMode_avg'] = Total_pivot[ESMode_columns].mean(axis=1)
Total_pivot['ESMode_sum'] = Total_pivot[ESMode_columns].sum(axis=1)

# 将特征与标签组合
train_data = pd.merge(energy_data,Total_pivot, on=['Time', 'BS'], how='left')
test_data = pd.merge(power_consumption_prediction, Total_pivot, on=['Time', 'BS'], how='left')
combined_data = pd.concat([train_data, test_data], ignore_index=False)


# 对时间和基站信息进行独热码编码
combined_data['Time'] = pd.to_datetime(combined_data['Time']).dt.dayofyear*24+pd.to_datetime(combined_data['Time']).dt.hour 
combined_data = pd.get_dummies(combined_data, columns=['Time','BS'])

# 分割训练集与测试集
train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

# print(len(train_data))
# print(len(combined_data))

# 删除标准差为0的特征
def preprocess_data(df, columns_with_std_zero):
    X = df.drop(columns=['Energy'] + columns_with_std_zero)
    y = df['Energy']
    return X, y

columns_with_std_zero = train_data.columns[train_data.std() == 0].tolist()

# 将训练集和测试集分为特征与标签
X,y = preprocess_data(train_data,columns_with_std_zero)
X_test,y_test = preprocess_data(test_data,columns_with_std_zero)

# print("X:\n",X)
# print("y:\n",y)
# print("X_test:\n",X_test)
# print("y_test:\n",y_test)

In [ ]:
# 定义损失函数
def MAPELoss(output, target):
    return torch.mean(torch.abs((target - output) / target))

def calculate_mape(y_true, y_pred):
    y_true = y_true.cpu().detach().numpy()
    y_pred = y_pred.cpu().detach().numpy()
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_wmape(y_true,y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

def WMAPELoss(y_pred, y_true, weights):
    return torch.sum(weights * torch.abs(y_true - y_pred)) / torch.sum(weights)

# 定义模型
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.fc(x)
        return x * y

class LinearSEBlock(nn.Module):
    def __init__(self, input_dim, output_dim, reduction=16, activation="leakyrelu", use_bn=True, use_se=True):
        super(LinearSEBlock, self).__init__()
        
        # 线性块
        self.fc = nn.Linear(input_dim, output_dim)
        
        
        if use_bn:
            self.bn = nn.BatchNorm1d(output_dim)
        else:
            self.bn = None
        
        if activation == "leakyrelu":
            self.act = nn.LeakyReLU()
        elif activation == "relu":
            self.act = nn.ReLU()
        
        # SE层
        if use_se:
            self.se = SELayer(output_dim, reduction)
        else:
            self.se = None

    def forward(self, x):
        x = self.fc(x)
        if self.bn:
            x = self.bn(x)
        x = self.act(x)
        if self.se:
            x = self.se(x)
        return x

class MyModel_attention(nn.Module):
    def __init__(self, input_dim, layers_dims=[512, 256, 128, 64], use_se_layers=[], **kwargs):
        super(MyModel_attention, self).__init__()
        dims = [input_dim] + layers_dims

        self.blocks = nn.ModuleList()
        
        if 0 in use_se_layers:
            se_kwargs = {key: kwargs[key] for key in kwargs if key not in ['activation', 'use_bn']}
            self.input_se = SELayer(input_dim, **se_kwargs)
            use_se_layers.remove(0)  
        else:
            self.input_se = None

        for i in range(len(dims)-1):
            self.blocks.append(LinearSEBlock(dims[i], dims[i+1], use_se=(i+1 in use_se_layers), **kwargs))
        
        self.fc_final = nn.Linear(layers_dims[-1], 1)
        
    def forward(self, x):
        if self.input_se:
            x = self.input_se(x)
        for block in self.blocks:
            x = block(x)
        x = self.fc_final(x)
        return x
        
def weight_init(m):
    if isinstance(m, torch.nn.Conv1d):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
    elif isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

## Train ##

In [ ]:
# 分割训练集与测试集
X_tensor = torch.Tensor(X.values.astype(np.float32)).to(device)
y_tensor = torch.Tensor(y.values.astype(np.float32)).unsqueeze(1).to(device)
X_data = TensorDataset(X_tensor, y_tensor)
X_loader = DataLoader(dataset=X_data, batch_size=2**12, shuffle=True)
X_test_tensor = torch.Tensor(X_test.values.astype(np.float32)).to(device)
y_test_tensor = torch.Tensor(y_test.values.astype(np.float32)).unsqueeze(1).to(device)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=8192, shuffle=False)

# 定义模型参数
continue_train = 0
input_dim = X.shape[1]
layers_dims = [1024, 512, 256, 128, 64]
use_se_layers = []
reduction = 16
activation = "leakyrelu"
use_bn = True
num_epochs = 2000
filename = f"./Energy_Consumption_Model.pth"

In [ ]:
import random

# 设置随机数
seed = 2024
random.seed(seed)  
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# 设定选用的模型
if continue_train == 1:
    model = torch.load(filename)

else:
    model = MyModel_attention(input_dim=input_dim, layers_dims=layers_dims, use_se_layers=use_se_layers, reduction=reduction, activation=activation, use_bn=use_bn).to(device)

# 定义损失函数和优化器
criterion = MAPELoss
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=3000, gamma=0.1)  


# 定义加权平均绝对百分比误差 WMAPE计算函数
def calculate_WMAPE(outputs, labels):
    outputs_np = outputs.detach().cpu().numpy()
    labels_np= labels.detach().cpu().numpy()
    Top=0
    Bottom=0
    for i,pred in enumerate(outputs_np) :
        Top+=np.abs(labels_np[i]-pred)
        Bottom+=np.abs(labels_np[i])
    wmape=Top/Bottom
    return wmape.item()

# 训练模型
progress_bar_info = []
progress_WMAPE = []
pbar = tqdm(range(num_epochs),desc="Processing", ncols=150)
try:
    for epoch in pbar:
        start_time = time.time()
        
        running_loss = 0.0
        running_WMAPE = 0.0 

        model.train() 
        
        for i, data in enumerate(X_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            
            #反向传播 优化
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 计算累加损失和WMAPE
            running_loss += loss.item()*100
            running_WMAPE += calculate_WMAPE(outputs, labels)*100
        
        #计算WMAPE的平均值
        avg_WMAPE = running_WMAPE / (i + 1)
        scheduler.step()

        #计算每个epoch消耗的时间
        end_time = time.time()
        epoch_duration = end_time - start_time

        #添加损失函数信息和WMAPE信息
        postfix_str = "Loss: {:.4f}, Epoch Duration: {:.2f} seconds".format(running_loss / (i+1), epoch_duration) # i+1 == len(X_loader)
        progress_bar_info.append(postfix_str)
        WMAPE_str = "{:.2f}".format(avg_WMAPE)
        progress_WMAPE.append(WMAPE_str)
        
        pbar.set_postfix_str(postfix_str+",WMAPE:{:.2f}".format(avg_WMAPE))
        if  running_loss / len(X_loader) <= 0.23:
            break
        
    torch.save(model, filename)
    print(f"模型保存为: {filename}")
    print('训练完成')

except KeyboardInterrupt:
    torch.save(model, filename)
    print(f"模型保存为: {filename}")

# 创建进度信息文件的文件名
progress_filename = filename.replace(".pth", "_progress.txt")
with open(f"./{progress_filename}", "w") as f:
    for line in progress_bar_info:
        f.write(line + "\n")

# 创建WMAPE.txt文件
WMAPE_filename= filename.replace(".pth", "_WMAPE.txt")
with open(f"./{WMAPE_filename}", "w") as f:
    for line in progress_WMAPE:
        f.write(line + "\n")


## Evaluate ##

In [ ]:
#若为1 则加载最佳模型;若为0 则加载刚刚训练的模型
evaluate_flag = 0

if evaluate_flag == 1:
    model = torch.load("BestModel/Energy_Consumption_Model.pth",map_location=torch.device('cpu'))
else:
    model = torch.load("./Energy_Consumption_Model.pth",map_location=torch.device('cpu'))


model.eval()
# 预测能耗
predictions = []
with torch.no_grad(): 
    for i, data in enumerate(test_loader, 0):
        inputs, _ = data
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())

#预测能耗的1xn多维数组
predictions_numpy = np.array(predictions)

#将多维数组降维成一维数组
predictions_flatten = predictions_numpy.flatten()
predictions_series = pd.Series(predictions_flatten, name="PredictedEnergy")

#将功耗预测结果填充进待预测数据中
test_data_new = pd.read_csv('EnergyContest/power_consumption_prediction.csv')
test_data_new['Energy'] = predictions_series
test_data_new['ID'] = test_data_new['Time'].astype(str) + "_" + test_data_new['BS'].astype(str)
test_data_new = test_data_new[['ID', 'Energy']]
test_data_new.to_csv('./AI预测结果.csv', index=False)